In [ ]:
# KNN

import torch
import numpy as np
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import tensorflow as tf
import time

# Define the data structure
class MyDataset(Dataset):
    def __init__(self,filepath):
        xy = np.loadtxt(filepath,delimiter=',',dtype=np.float32)
        self.len = xy.shape[0]
        
        # load the data after the second column of the dataset (RSS data)
        self.x_data = torch.from_numpy(xy[:,2:])
        # load the first two columns of the dataset (Coordinate)
        self.y_data = torch.from_numpy(xy[:,0:2])
        
        #print("The data has prepared...")
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.len

# Initialize the positioning error
error_0 = []

for z in range(1):
    print(z)
    # Settings
    batch_size = 14
    
    # load traning and testing dataset
    train_file = "./dataset/03_14_train.csv"
    test_file = "./dataset/03_10_test.csv"
    mydataset_train = MyDataset(train_file)
    mydataset_test = MyDataset(test_file)
    train_loader = DataLoader(dataset=mydataset_train,
                             batch_size=batch_size,
                             shuffle=True,
                             num_workers=2)
    test_loader = DataLoader(dataset=mydataset_test,
                             batch_size=10,
                             shuffle=True,
                             num_workers=2)

    start_time = time.time()
    
    # Data preparing
    for data, labels in train_loader:
        train_data = data.view(-1, 64).numpy()
        train_crd = labels.view(-1,2).numpy()
        train_crd = 30*train_crd
    for data, labels in test_loader:
        test_data = data.view(-1, 64).numpy() 
        test_crd = labels.view(-1,2).numpy()
        test_crd = 30*test_crd
        
    # 1.Define the accuracy function
    def getAccuracy(pred_crd,test_crd):
        return np.mean(np.sqrt(np.sum((pred_crd-test_crd)**2,1)))
    
    # 2.Using KNN module
    from sklearn.neighbors import KNeighborsRegressor
    knn = KNeighborsRegressor(4, weights='uniform', metric='euclidean')
    %time knn.fit(train_data, train_crd)
    %time pred_crd = knn.predict(test_data)
    accuracy = getAccuracy(pred_crd, test_crd)
    print("accuracy: ", accuracy, "m")    

    print("--- Run time: %s mins ---" % np.round(((time.time() - start_time)/60),4))
    #error_0.append(e)
